In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Загрузка данных
data = pd.read_csv('../datasets/diabetes.csv')

In [2]:
print(data)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                  

In [3]:
import numpy as np

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [4]:
# Разделение данных на признаки и целевую переменную
X_train = train_data.drop('Outcome', axis=1)
y_train = train_data['Outcome']
X_test = test_data.drop('Outcome', axis=1)
y_test = test_data['Outcome']

In [5]:
# Вычисление математического ожидания и дисперсии для каждого класса
means = X_train.groupby(y_train).mean()
variances = X_train.groupby(y_train).var()


In [6]:
print(means)
print(variances)

         Pregnancies     Glucose  BloodPressure  SkinThickness    Insulin  \
Outcome                                                                     
0           3.241895  110.214464      68.309227      19.748130  72.254364   
1           4.685446  140.887324      71.497653      21.624413  98.727700   

               BMI  DiabetesPedigreeFunction        Age  
Outcome                                                  
0        30.256608                  0.431254  30.556110  
1        35.234272                  0.540545  37.333333  
         Pregnancies      Glucose  BloodPressure  SkinThickness       Insulin  \
Outcome                                                                         
0           9.158840   678.328890     312.044140     212.618903  10233.220137   
1          13.093985  1070.185357     395.534193     285.301665  19298.010408   

               BMI  DiabetesPedigreeFunction         Age  
Outcome                                                   
0        56.8343

In [27]:
def product(iterable):
    result = 1
    for x in iterable:
        result *= x
    return result

def gaussian_naive_bayes(X, means, variances):
    likelihoods = [
        (2.71828 ** (-((x - mean)**2) / (2 * variance))) / ((2 * 3.14159 * variance) ** 0.5)
        for x, mean, variance in zip(X, means, variances)
    ]
    return product(likelihoods)




In [31]:
predictions = []
for index, row in X_test.iterrows():
    posteriors = []
    for outcome in means.index:
        prior = len(y_train[y_train == outcome]) / len(y_train)
        likelihood = gaussian_naive_bayes(row, means.loc[outcome], variances.loc[outcome])
        posterior = prior * np.sum(np.log(likelihood))  # используем np.sum вместо np.prod
        posteriors.append(posterior)
    predictions.append(np.argmax(posteriors))


In [32]:
# Вычисление точности
accuracy = accuracy_score(y_test, predictions)
print(f'Точность наивного байесовского классификатора по гаусу: {accuracy:.2f}')

Точность наивного байесовского классификатора: 0.36


In [39]:
from collections import defaultdict
from math import exp, sqrt, pi, log

def calculate_prior(y):
    class_counts = defaultdict(int)
    for label in y:
        class_counts[label] += 1
    total_samples = len(y)
    prior = {label: count / total_samples for label, count in class_counts.items()}
    return prior

def calculate_likelihood(X, means, variances):
    likelihoods = []
    for i in range(len(means)):
        exponent = exp(-(X.iloc[i] - means[i])**2 / (2 * variances[i]))
        likelihood = (1 / (sqrt(2 * pi * variances[i]))) * exponent
        likelihoods.append(likelihood)
    return likelihoods

def fit(X, y):
    classes = list(set(y))
    class_means = {label: [] for label in classes}
    class_variances = {label: [] for label in classes}

    for label in classes:
        class_data = X[y == label]
        class_means[label] = class_data.mean()
        class_variances[label] = class_data.var()

    return class_means, class_variances, calculate_prior(y)

def predict(sample, class_means, class_variances, prior):
    posteriors = {}
    for label, prior_prob in prior.items():
        means = class_means[label]
        variances = class_variances[label]
        likelihoods = calculate_likelihood(sample, means, variances)
        posterior = prior_prob * exp(sum([log(likelihood) for likelihood in likelihoods]))
        posteriors[label] = posterior

    return max(posteriors, key=posteriors.get)

# Используем данные
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

X_train = train_data.drop('Outcome', axis=1)
y_train = train_data['Outcome']
X_test = test_data.drop('Outcome', axis=1)
y_test = test_data['Outcome']

class_means, class_variances, prior = fit(X_train, y_train)

# Предсказание на тестовых данных
predictions = [predict(sample, class_means, class_variances, prior) for _, sample in X_test.iterrows()]

# Вычисление точности
accuracy = accuracy_score(y_test, predictions)
print(f'Точность наивного байесовского классификатора: {accuracy:.2f}')

Точность наивного байесовского классификатора: 0.77


C:\Users\Redmi\AppData\Local\Temp\ipykernel_9556\1870624265.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exponent = exp(-(X.iloc[i] - means[i])**2 / (2 * variances[i]))
C:\Users\Redmi\AppData\Local\Temp\ipykernel_9556\1870624265.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  likelihood = (1 / (sqrt(2 * pi * variances[i]))) * exponent


1. **`calculate_prior(y)`**:

   Априорная вероятность для класса \(c\) вычисляется как:

   \[ P(Y = c) = \frac{\text{Количество образцов класса } c}{\text{Общее количество образцов}} \]

   где \(Y\) - целевая переменная (класс), \(c\) - конкретный класс.

2. **`calculate_likelihood(X, means, variances)`**:

   Вероятность для каждого признака \(X_i\) в классе \(c\) вычисляется с использованием нормального распределения:

   \[ P(X_i \mid Y = c) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(X_i - \mu)^2}{2\sigma^2}} \]

   где \(\mu\) - среднее значение, \(\sigma^2\) - дисперсия для признака \(X_i\) в классе \(c\).

3. **`fit(X, y)`**:

   В этой функции вычисляются средние \(\mu\) и дисперсии \(\sigma^2\) для каждого признака \(X_i\) и каждого класса \(c\).

4. **`predict(sample, class_means, class_variances, prior)`**:

   Апостериорная вероятность для класса \(c\) при заданном образце \(X\) вычисляется как:

   \[ P(Y = c \mid X) \propto P(Y = c) \prod_{i=1}^{n} P(X_i \mid Y = c) \]

   где \(n\) - количество признаков.

   В данной реализации используется логарифмирование для избежания численных проблем:

   \[ \log P(Y = c \mid X) \propto \log P(Y = c) + \sum_{i=1}^{n} \log P(X_i \mid Y = c) \]

5. **Использование данных**:

   Здесь данные разделяются на обучающую и тестовую выборки. Обучается модель с использованием `fit`, а затем делаются предсказания с использованием `predict`. Вычисляется точность предсказаний с помощью `accuracy_score`.

Общий принцип наивного байесовского классификатора заключается в предположении о независимости признаков при заданном классе и использовании этого предположения для вычисления апостериорных вероятностей. В данном случае, используются нормальные распределения для вычисления вероятностей.


### Гауссовский наивный байесовский классификатор (Gaussian Naive Bayes):

Тип данных: Подходит для непрерывных числовых данных.
Предположение: Предполагает, что каждый признак имеет нормальное распределение внутри каждого класса.
Формула: Использует плотность вероятности нормального распределения для оценки вероятности каждого значения признака для каждого класса.
Примеры применения: Когда признаки в данных представляют собой непрерывные числовые значения, такие как рост, вес, температура и т. д.


### Мультиномиальный наивный байесовский классификатор (Multinomial Naive Bayes):

Тип данных: Подходит для данных с дискретными признаками, как правило, это частоты или счетчики.
Предположение: Предполагает, что данные представляют собой распределение категорий.
Формула: Использует мультиномиальное распределение для вычисления вероятности каждого значения признака для каждого класса.
Примеры применения: Когда признаки представляют собой дискретные значения, такие как количество слов в документе, категории товаров и т. д.